In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.decomposition import PCA
from tensorflow.keras import layers, models, backend as K
%matplotlib inline
plt.rcParams['font.sans-serif'] = ['SimHei']  # 中文显示
plt.rcParams['axes.unicode_minus'] = False

In [3]:
# ========== 1. 数据加载与预处理 ==========
def load_data(file_path):
    """加载并预处理CMAPSS数据集"""
    # 定义完整的列名（根据PHM08数据集规范）
    column_names = [
        'unit', 'time', 
        'op_setting_1', 'op_setting_2', 'op_setting_3',
        'sm_1', 'sm_2', 'sm_3', 'sm_4', 'sm_5', 'sm_6',
        'sm_7', 'sm_8', 'sm_9', 'sm_10', 'sm_11', 'sm_12',
        'sm_13', 'sm_14', 'sm_15', 'sm_16', 'sm_17', 'sm_18',
        'sm_19', 'sm_20', 'sm_21'
    ]
    
    # 读取数据并清理
    df = pd.read_csv(file_path, sep=' ', header=None)
    df.dropna(axis=1, how='all', inplace=True)
    df.columns = column_names[:df.shape[1]]
    return df

In [4]:
def select_and_scale_sensors(df, selected_sensors):
    """选择并预处理传感器数据"""
    # 验证传感器存在性
    missing = [s for s in selected_sensors if s not in df.columns]
    if missing:
        raise ValueError(f"缺少传感器: {missing}")
    
    # 按发动机分组处理
    engines = []
    for unit_id in df['unit'].unique():
        engine_df = df[df['unit'] == unit_id]
        
        # 提取并缩放传感器数据
        sensor_data = engine_df[selected_sensors].values
        scaled_data = MinMaxScaler(feature_range=(-1, 1)).fit_transform(sensor_data)
        
        engines.append(scaled_data)
    
    print(f"成功预处理 {len(engines)} 台发动机数据")
    print(f"使用的传感器: {selected_sensors}")
    return engines

In [5]:
# ========== 2. VAE模型构建 ==========
class VAEDetector:
    """变分自编码器异常检测器"""
    def __init__(self, input_dim, latent_dim=8):
        self.input_dim = input_dim
        self.latent_dim = latent_dim
        self._build_model()
    
    def _build_model(self):
        # 编码器
        inputs = layers.Input(shape=(self.input_dim,))
        x = layers.Dense(32, activation='relu')(inputs)
        x = layers.Dense(16, activation='relu')(x)
        
        # 潜在空间
        z_mean = layers.Dense(self.latent_dim)(x)
        z_log_var = layers.Dense(self.latent_dim)(x)
        
        # 重参数化
        def sampling(args):
            z_mean, z_log_var = args
            epsilon = K.random_normal(shape=(K.shape(z_mean)[0], self.latent_dim))
            return z_mean + K.exp(0.5 * z_log_var) * epsilon
            
        z = layers.Lambda(sampling)([z_mean, z_log_var])
        
        # 解码器
        decoder_input = layers.Input(shape=(self.latent_dim,))
        d = layers.Dense(16, activation='relu')(decoder_input)
        d = layers.Dense(32, activation='relu')(d)
        outputs = layers.Dense(self.input_dim, activation='tanh')(d)
        
        # 编译模型
        self.encoder = models.Model(inputs, z_mean)
        self.decoder = models.Model(decoder_input, outputs)
        
        vae_outputs = self.decoder(z)
        self.vae = models.Model(inputs, vae_outputs)
        
        # 自定义损失
        reconstruction_loss = K.mean(K.square(inputs - vae_outputs), axis=-1)
        kl_loss = -0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
        self.vae.add_loss(K.mean(reconstruction_loss + kl_loss))
        self.vae.compile(optimizer='adam')

In [6]:
# ========== 3. 异常检测 ==========
def find_anomaly_point(engine_data, model, smooth_window=10, detect_window_ratio=0.1, sigma=3):
    """检测异常起始点"""
    # 计算重构误差
    reconstructions = model.vae.predict(engine_data)
    mse = np.mean(np.square(engine_data - reconstructions), axis=1)
    
    # 滑动平均
    window = np.ones(smooth_window)/smooth_window
    smoothed = np.convolve(mse, window, mode='valid')
    
    # 动态阈值
    baseline = smoothed[:int(len(smoothed)*0.25)]
    upper = baseline.mean() + sigma * baseline.std()
    lower = baseline.mean() - sigma * baseline.std()
    
    # 检测持续异常
    detect_window = max(int(len(engine_data)*detect_window_ratio), 1)
    exceed = np.where((smoothed > upper) | (smoothed < lower))[0]
    
    for idx in exceed:
        end = idx + detect_window
        if end >= len(smoothed):
            continue
        if (smoothed[idx:end] > upper).all() or (smoothed[idx:end] < lower).all():
            return idx + smooth_window - 1  # 转回原始索引
    
    return len(engine_data)

In [7]:
# ========== 4. 健康指标生成 ==========
def generate_health_indicator(engine_data, model, selected_sensors, detect_params):
    """生成健康指标曲线"""
    # 检测异常点
    anomaly_point = find_anomaly_point(engine_data, model, **detect_params)
    total_cycles = len(engine_data)
    
    # 初始化HI
    HI = np.ones(total_cycles)
    
    if anomaly_point < total_cycles - 10:  # 需要足够的数据进行退化建模
        # PCA特征融合
        scaler = StandardScaler().fit(engine_data[:anomaly_point, selected_sensors])
        normalized_data = scaler.transform(engine_data[:, selected_sensors])
        
        pca = PCA(n_components=1).fit(normalized_data[:anomaly_point])
        pca_scores = pca.transform(normalized_data).flatten()
        
        # 确保退化方向正确
        if np.corrcoef(pca_scores[anomaly_point:], np.arange(len(pca_scores[anomaly_point:])))[0,1] < 0:
            pca_scores = -pca_scores
        
        # 归一化处理
        hi_degradation = MinMaxScaler().fit_transform(pca_scores.reshape(-1,1)).flatten()
        
        # 应用指数平滑
        alpha = 0.3
        smoothed_hi = hi_degradation.copy()
        for i in range(1, len(smoothed_hi)):
            smoothed_hi[i] = alpha*hi_degradation[i] + (1-alpha)*smoothed_hi[i-1]
        
        # 调整范围到[1,0]
        HI = 1 - (smoothed_hi - smoothed_hi.min())/(smoothed_hi.max() - smoothed_hi.min())
    
    return HI, anomaly_point

In [8]:
# ========== 5. 可视化 ==========
def plot_health_status(engine_data, HI, anomaly_point, sensor_names):
    """综合可视化健康状态"""
    plt.figure(figsize=(15, 6))
    
    # 传感器数据
    plt.subplot(121)
    for i in range(3):  # 显示前3个传感器
        plt.plot(engine_data[:, i], label=sensor_names[i], alpha=0.7)
    plt.axvline(anomaly_point, color='r', linestyle='--', linewidth=2)
    plt.title('传感器数据监测')
    plt.xlabel('运行周期')
    plt.ylabel('归一化值')
    plt.legend()
    
    # 健康指标
    plt.subplot(122)
    plt.plot(HI, color='darkorange', linewidth=2)
    plt.axvline(anomaly_point, color='r', linestyle='--', label=f'异常起始点 ({anomaly_point})')
    plt.ylim(-0.1, 1.1)
    plt.title('健康指标 (HI) 退化曲线')
    plt.xlabel('运行周期')
    plt.ylabel('健康指标')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

In [9]:
# 配置参数
SELECTED_SENSORS = ['sm_2', 'sm_3', 'sm_4', 'sm_7', 'sm_8', 'sm_9',
                       'sm_11', 'sm_12', 'sm_13', 'sm_14', 'sm_15', 'sm_17', 'sm_20', 'sm_21']
DETECT_PARAMS = {
        'smooth_window': 10,
        'detect_window_ratio': 0.1,
        'sigma': 3
    }
VAE_PARAMS = {
        'input_dim': len(SELECTED_SENSORS),
        'latent_dim': 8
    }

In [10]:
# 1. 加载数据
print("正在加载数据...")
df = load_data("data/train_FD001.txt")
engines = select_and_scale_sensors(df, SELECTED_SENSORS)

正在加载数据...
成功预处理 100 台发动机数据
使用的传感器: ['sm_2', 'sm_3', 'sm_4', 'sm_7', 'sm_8', 'sm_9', 'sm_11', 'sm_12', 'sm_13', 'sm_14', 'sm_15', 'sm_17', 'sm_20', 'sm_21']


In [11]:
# 2. 训练VAE
print("\n训练VAE模型中...")
detector = VAEDetector(**VAE_PARAMS)
train_data = np.vstack(engines)
detector.vae.fit(train_data, epochs=100, batch_size=32, verbose=1)   


训练VAE模型中...
Epoch 1/100
645/645 [==============================] - 2s 1ms/step - loss: 0.1943
Epoch 2/100
645/645 [==============================] - 1s 1ms/step - loss: 0.1766
Epoch 3/100
645/645 [==============================] - 1s 1ms/step - loss: 0.1758
Epoch 4/100
645/645 [==============================] - 1s 1ms/step - loss: 0.1754
Epoch 5/100
645/645 [==============================] - 1s 1ms/step - loss: 0.1754
Epoch 6/100
645/645 [==============================] - 1s 1ms/step - loss: 0.1753
Epoch 7/100
645/645 [==============================] - 1s 1ms/step - loss: 0.1752
Epoch 8/100
645/645 [==============================] - 1s 2ms/step - loss: 0.1752
Epoch 9/100
645/645 [==============================] - 1s 1ms/step - loss: 0.1752
Epoch 10/100
645/645 [==============================] - 1s 1ms/step - loss: 0.1752
Epoch 11/100
645/645 [==============================] - 1s 1ms/step - loss: 0.1751
Epoch 12/100
645/645 [==============================] - 1s 1ms/step - loss: 0.1751


In [12]:
# 3. 生成健康指标
sensor_indices = [df.columns.get_loc(s) for s in SELECTED_SENSORS]
engine_id = 0  # 选择第一台发动机演示
    
HI, anomaly = generate_health_indicator(
        engines[engine_id], 
        detector,
        sensor_indices,
        DETECT_PARAMS
    )   

6/6 [==============================] - 0s 2ms/step


IndexError: index 15 is out of bounds for axis 1 with size 14

In [ ]:
# 4. 可视化结果
print(f"\n发动机 {engine_id+1} 分析结果：")
print(f"- 总运行周期: {len(engines[engine_id])}")
print(f"- 异常起始点: {anomaly}")
print(f"- 剩余使用寿命: {len(engines[engine_id]) - anomaly} 周期")

In [ ]:
plot_health_status(engines[engine_id], HI, anomaly, SELECTED_SENSORS[:3])